# **Download rdkit và mordred**

In [ ]:
!pip install rdkit-pypi
!pip install mordred

# **Import thư viện**

In [ ]:
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from mordred import Calculator, descriptors

# **Đọc file data từ link**

In [ ]:
url = 'https://raw.githubusercontent.com/jwxia2014/HDAC3i-Finder/master/data_for_modeling/HDAC3_train.csv'
url_test = 'https://raw.githubusercontent.com/jwxia2014/HDAC3i-Finder/master/data_for_modeling/HDAC3_test.csv'
dataset = pd.read_csv(url)
dataset_test = pd.read_csv(url_test)


In [ ]:
# truy cập kích thước dataset_train
dataset.shape

(790, 4)

# **Xem DataFrame 5 hàng đầu tiên**

In [ ]:
# truy cập kích thước dataset_test
dataset_test.shape

(198, 4)

In [ ]:
dataset.head()

,ID,SMILES,IC50_mean,Type
0,CHEMBL3890451,Cc1csc(C(=O)NCCCCCC(=O)Nc2ccccc2N)n1,0.1290,1
1,CHEMBL3953948,Nc1ccccc1NC(=O)CCCCCNC(=O)c1ccc[nH]1,0.4600,1
2,CHEMBL3233703,O=C(/C=C/c1ccc(OC[C@H](Cc2c[nH]c3ccccc23)NC(=O...,0.0055,1
3,CHEMBL3899528,Nc1ccccc1NC(=O)CCCCCNC(=O)c1ccccc1OC(F)F,0.2140,1
4,CHEMBL178456,O=C(CS)NCCCCCC(=O)Nc1cccc2cccnc12,1.3200,0


In [ ]:
dataset_test.head()

,ID,SMILES,IC50_mean,Type
0,CHEMBL1088734,O=C(CCCCCCC(=O)Nc1cccc(-c2cn(-c3ccc(C(F)(F)F)c...,0.0034,1
1,CHEMBL1088736,O=C(CCCCCCC(=O)Nc1cccc(-c2cn(-c3cc(CO)cc(CO)c3...,0.0033,1
2,CHEMBL1091475,O=C(CCCCCCC(=O)Nc1cccc(-c2cn(-c3ccc(I)cc3)nn2)...,0.0030,1
3,CHEMBL1092762,Cc1ccc(NC(=O)CCCCC(=O)NO)cc1,0.1140,1
4,CHEMBL1093042,CCc1ccc(NC(=O)CCCCC(=O)NO)cc1,0.2400,1


# **Chuẩn hoá các chuỗi SMILES**

In [ ]:
def canonical_smiles(smiles):
    # Tạo danh sách đối tượng mol từ danh sách các chuỗi SMILES đầu vào
    mols = [Chem.MolFromSmiles(smi) for smi in smiles]
    # chuyển đổi đối tượng 'mol' thành 1 chuỗi Canonical SMILES tương ứng
    smiles = [Chem.MolToSmiles(mol) for mol in mols]
    # Trả về danh sách các chuỗi Canonical SMILES
    return smiles

Tính toán ra số chuỗi canonical smiles

In [ ]:
# Canonical SMILES
Canon_train = canonical_smiles(dataset.SMILES)
Canon_test = canonical_smiles(dataset_test.SMILES)

In [ ]:
len(Canon_train)

790

In [ ]:
len(Canon_test)

198

Gán canon_smiles cho cột smiles của dataframe

In [ ]:
dataset['SMILES'] = Canon_train
# in ra dataset sau khi gán
dataset

,ID,SMILES,IC50_mean,Type
0,CHEMBL3890451,Cc1csc(C(=O)NCCCCCC(=O)Nc2ccccc2N)n1,0.12900,1
1,CHEMBL3953948,Nc1ccccc1NC(=O)CCCCCNC(=O)c1ccc[nH]1,0.46000,1
2,CHEMBL3233703,O=C(/C=C/c1ccc(OC[C@H](Cc2c[nH]c3ccccc23)NC(=O...,0.00550,1
3,CHEMBL3899528,Nc1ccccc1NC(=O)CCCCCNC(=O)c1ccccc1OC(F)F,0.21400,1
4,CHEMBL178456,O=C(CS)NCCCCCC(=O)Nc1cccc2cccnc12,1.32000,0
...,...,...,...,...
785,CHEMBL3652242,O=C(NO)c1cnc(Nc2cccc(C(F)(F)F)c2)nc1,0.31300,1
786,CHEMBL3601775,COc1ccc(Cn2ccc3ccc(NC(=O)CCCCCCC(=O)NO)cc3c2=O...,0.10770,1
787,CHEMBL3356937,Cc1cccc(CCNC(=O)[C@H](CCCCCS)NC(=O)[C@@H]2CCCC...,0.12200,1
788,CHEMBL4063718,Cc1ccc(F)c(Nc2ncnc3cc(NC(=O)CCCCCCC(=O)NO)sc23)c1,0.06754,1


In [ ]:
dataset_test['SMILES'] = Canon_test
# in ra dataset sau khi gán
dataset_test

,ID,SMILES,IC50_mean,Type
0,CHEMBL1088734,O=C(CCCCCCC(=O)Nc1cccc(-c2cn(-c3ccc(C(F)(F)F)c...,0.0034,1
1,CHEMBL1088736,O=C(CCCCCCC(=O)Nc1cccc(-c2cn(-c3cc(CO)cc(CO)c3...,0.0033,1
2,CHEMBL1091475,O=C(CCCCCCC(=O)Nc1cccc(-c2cn(-c3ccc(I)cc3)nn2)...,0.0030,1
3,CHEMBL1092762,Cc1ccc(NC(=O)CCCCC(=O)NO)cc1,0.1140,1
4,CHEMBL1093042,CCc1ccc(NC(=O)CCCCC(=O)NO)cc1,0.2400,1
...,...,...,...,...
193,CHEMBL484073,O=C(/C=C/c1ccc2cccnc2c1)NCCCCCC(=O)NO,0.0067,1
194,CHEMBL488747,O=C(CCCCCCC(=O)Nc1cccc(-c2cnnn2Cc2ccccc2)c1)NO,0.0045,1
195,CHEMBL504161,O=C(c1ccc(-c2nc(Cc3cccs3)no2)s1)C(F)(F)F,0.6300,1
196,CHEMBL514304,Nc1ccc(-c2ccsc2)cc1NC(=O)c1ccc(CN2CCC3(CCCN3)C...,8.7060,0


# **Loại bỏ trùng lặp**

Train Set

In [ ]:
# trả về dataframe con có chứa các chuỗi smiles trùng lặp
duplicates_smiles = dataset[dataset['SMILES'].duplicated()]['SMILES'].values
len(duplicates_smiles)

0

In [ ]:
# dataframe con chứa các chuỗi smiles trùng lặp được sắp xếp từ nhỏ đến lớn theo mã ASCII
dataset[dataset['SMILES'].isin(duplicates_smiles)].sort_values(by=['SMILES'])

,ID,SMILES,IC50_mean,Type


In [ ]:
# bỏ các giá trị smiles trùng lặp
dataset_TrainNew = dataset.drop_duplicates(subset=['SMILES'])
# số lượng dòng trong dataset mới
len(dataset_TrainNew)

790

Test Set

In [ ]:
duplicates_smiles = dataset_test[dataset_test['SMILES'].duplicated()]['SMILES'].values
len(duplicates_smiles)

0

In [ ]:
dataset_test[dataset_test['SMILES'].isin(duplicates_smiles)].sort_values(by=['SMILES'])

,ID,SMILES,IC50_mean,Type


In [ ]:
dataset_TestNew = dataset_test.drop_duplicates(subset=['SMILES'])
len(dataset_TestNew)

198

# **Tính toán các bộ mô tả**

## Tính toán mô tả phần tử sử dụng thư viện mordred

In [ ]:
def All_Mordred_descriptors(data):
    calc = Calculator(descriptors, ignore_3D=False)
    mols = [Chem.MolFromSmiles(smi) for smi in data]

    # pandas df
    df = calc.pandas(mols)
    return df

### Train set

In [ ]:
All_Mordred_descriptors_train = All_Mordred_descriptors(dataset_TrainNew['SMILES'])
All_Mordred_descriptors_train.shape

In [ ]:
All_Mordred_descriptors_train

## Test set

In [ ]:
All_Mordred_descriptors_test = All_Mordred_descriptors(dataset_TestNew['SMILES'])
All_Mordred_descriptors_test.shape

In [ ]:
All_Mordred_descriptors_test

## Tính toán mô tả phần tử sử dụng thư viện RDkit

In [ ]:
def RDkit_descriptors(smiles):
    # chuyển đổi chuỗi SMILES (Simplified Molecular Input Line Entry System) thành một đối tượng phân tử (molecule)
    mols = [Chem.MolFromSmiles(i) for i in smiles]
    # tính toán các đặc trưng hóa học. Các đặc trưng này được liệt kê trong _descList từ thư viện Descriptors
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    # Xác định danh sách tên của các đặc trưng hóa học
    desc_names = calc.GetDescriptorNames()

    # Khởi tạo danh sách rỗng Mol_descriptors để lưu trữ các giá trị đặc trưng hóa học của từng phân tử.
    Mol_descriptors =[]
    for mol in mols:
        # add hydrogens to molecules
        mol=Chem.AddHs(mol)
        # Calculate all descriptors for each molecule (200 - 208)
        descriptors = calc.CalcDescriptors(mol)
        Mol_descriptors.append(descriptors)
    return Mol_descriptors,desc_names



### Train Set

In [ ]:
# Function call Train Set
Mol_descriptorsTrain,desc_names = RDkit_descriptors(dataset_TrainNew['SMILES'])

In [ ]:
df_with_200_descriptorsTrain = pd.DataFrame(Mol_descriptorsTrain,columns=desc_names)
df_with_200_descriptorsTrain

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,13.186860,-4.563393,13.186860,0.046277,0.505501,346.456,324.280,346.146347,128,0,...,0,0,0,0,0,1,0,0,0,0
1,13.171479,-4.590567,13.171479,0.170916,0.444747,314.389,292.213,314.174276,122,0,...,0,0,0,0,0,0,0,0,0,0
2,14.456229,-4.604861,14.456229,0.064992,0.165809,473.504,449.312,473.175084,178,0,...,0,0,0,0,0,0,0,0,0,0
3,13.446561,-5.027820,13.446561,0.520481,0.424202,391.418,368.234,391.170748,150,0,...,0,0,0,0,0,0,0,0,0,0
4,13.292357,-4.553125,13.292357,0.576533,0.514414,331.441,310.273,331.135448,122,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,13.203763,-5.250594,13.203763,0.109946,0.597881,298.224,289.152,298.067760,110,0,...,0,0,0,0,0,0,0,0,0,0
786,14.301030,-4.860071,14.301030,0.454809,0.205405,481.549,450.301,481.221286,186,0,...,0,0,0,0,0,0,0,0,0,0
787,14.432546,-5.552311,14.432546,0.901464,0.327859,419.591,386.327,419.224263,160,0,...,0,0,0,0,0,0,0,0,0,0
788,15.301391,-4.683353,15.301391,0.030696,0.204771,445.520,421.328,445.158389,164,0,...,0,0,0,0,0,0,0,1,0,0


### Test Set

In [ ]:
# Function call Test Set
Mol_descriptorsTest,desc_names = RDkit_descriptors(dataset_TestNew['SMILES'])

In [ ]:
df_with_200_descriptorsTest = pd.DataFrame(Mol_descriptorsTest,columns=desc_names)
df_with_200_descriptorsTest

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,13.552246,-5.383452,13.552246,0.120995,0.223022,475.471,451.279,475.183124,180,0,...,0,0,0,0,0,0,0,0,0,0
1,13.685133,-4.792798,13.685133,0.145059,0.156112,467.526,438.294,467.216869,180,0,...,0,0,0,0,0,0,0,0,0,0
2,13.561140,-4.712758,13.561140,0.098274,0.155734,533.370,509.178,533.092388,162,0,...,0,0,0,0,0,0,0,0,0,0
3,12.801290,-4.338283,12.801290,0.667607,0.410259,250.298,232.154,250.131742,98,0,...,0,0,0,0,0,0,0,0,0,0
4,12.907586,-4.393758,12.907586,0.731701,0.401325,264.325,244.165,264.147393,104,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,12.968498,-4.507665,12.968498,0.551178,0.300469,327.384,306.216,327.158292,126,0,...,0,0,0,0,0,0,0,0,0,0
194,13.660713,-4.774356,13.660713,0.041256,0.247989,421.501,394.285,421.211390,162,0,...,0,0,0,0,0,0,0,0,0,0
195,12.643102,-5.242551,12.643102,0.146011,0.665900,344.339,337.283,343.990104,114,0,...,0,0,0,0,0,0,0,2,0,0
196,14.034483,-4.600579,14.034483,0.401343,0.484233,446.620,416.380,446.214033,166,0,...,0,0,0,0,0,0,0,1,0,0


## Tính toán mô tả phần tử sử dụng Morgan fingerprint (ECFP)

In [ ]:
def morgan_fpts(data):
    Morgan_fpts = []
    radius = 4
    for i in data:
        mol = Chem.MolFromSmiles(i)
        # 2 đại diện cho bán kính và 2048 đại diện số bit trong fingerprint
        fpts =  AllChem.GetMorganFingerprintAsBitVect(mol,radius,2048)
        mfpts = np.array(fpts)
        Morgan_fpts.append(mfpts)
    return np.array(Morgan_fpts)

### Train Set

In [ ]:
Morgan_fptsTrain = morgan_fpts(dataset_TrainNew['SMILES'])
Morgan_fptsTrain.shape

(790, 2048)

In [ ]:
Morgan_fptsTrain

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
# tạo ra 1 dataframe mới có tên Morgan_fingerprints từ ma trận fingerprint Morgan('Morgan_fpts')
Morgan_fingerprintsTrain = pd.DataFrame(Morgan_fptsTrain,columns=['Col_{}'.format(i) for i in range(Morgan_fptsTrain.shape[1])])
Morgan_fingerprintsTrain

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,...,Col_2038,Col_2039,Col_2040,Col_2041,Col_2042,Col_2043,Col_2044,Col_2045,Col_2046,Col_2047
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
786,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
787,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
788,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


###  Test Set

In [ ]:
Morgan_fptsTest = morgan_fpts(dataset_TestNew['SMILES'])
Morgan_fptsTest.shape

(198, 2048)

In [ ]:
Morgan_fptsTest

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [ ]:
# tạo ra 1 dataframe mới có tên Morgan_fingerprints từ ma trận fingerprint Morgan('Morgan_fpts')
Morgan_fingerprintsTest = pd.DataFrame(Morgan_fptsTest,columns=['Col_{}'.format(i) for i in range(Morgan_fptsTest.shape[1])])
Morgan_fingerprintsTest

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,...,Col_2038,Col_2039,Col_2040,Col_2041,Col_2042,Col_2043,Col_2044,Col_2045,Col_2046,Col_2047
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
194,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Tính toán mô tả phân tử sử dụng Avalon fingerprint

In [ ]:
from rdkit.Avalon import pyAvalonTools
def avalon_fingerprints(data):
    avalon_fpts = []
    for i in data:
        mol = Chem.MolFromSmiles(i)
        fpts =  pyAvalonTools.GetAvalonFP(mol, nBits=512)
        mfpts = np.array(fpts)
        avalon_fpts.append(mfpts)
    return np.array(avalon_fpts)

### Train Set

In [ ]:
avalonTrain = avalon_fingerprints(dataset_TrainNew['SMILES'])
avalonTrain.shape

(790, 512)

In [ ]:
avalonTrain

array([[1, 0, 0, ..., 0, 1, 1],
       [1, 0, 0, ..., 0, 1, 1],
       [1, 0, 0, ..., 0, 0, 1],
       ...,
       [1, 0, 0, ..., 0, 1, 0],
       [1, 1, 0, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 1, 0]])

In [ ]:
# Hiển thị dạng bảng
avalon_fingerprintsTrain = pd.DataFrame(avalonTrain,columns=['Col_{}'.format(i) for i in range(avalonTrain.shape[1])])
avalon_fingerprintsTrain

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,...,Col_502,Col_503,Col_504,Col_505,Col_506,Col_507,Col_508,Col_509,Col_510,Col_511
0,1,0,0,1,0,1,0,0,0,1,...,0,0,1,0,0,1,0,0,1,1
1,1,0,0,0,0,1,1,0,0,1,...,0,0,0,1,0,1,0,0,1,1
2,1,0,0,0,0,1,1,1,0,1,...,1,0,0,1,0,0,1,0,0,1
3,1,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
4,0,0,0,0,0,0,0,1,0,1,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,0,0,0,0,1,0,0,0,0,1,...,0,0,0,1,0,0,1,1,0,0
786,0,1,1,0,0,0,0,0,0,1,...,1,0,1,1,1,0,0,0,1,0
787,1,0,0,0,1,0,0,1,0,0,...,1,0,1,0,0,0,0,0,1,0
788,1,1,0,0,0,1,1,1,1,1,...,1,1,0,1,0,0,1,0,1,1


### Test Set

In [ ]:
avalonTest = avalon_fingerprints(dataset_TestNew['SMILES'])
avalonTest.shape

(198, 512)

In [ ]:
avalonTest

array([[0, 1, 0, ..., 1, 1, 0],
       [0, 1, 0, ..., 1, 1, 0],
       [0, 1, 0, ..., 1, 1, 0],
       ...,
       [1, 0, 0, ..., 0, 1, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 1, 1]])

In [ ]:
# Hiển thị dạng bảng
avalon_fingerprintsTest = pd.DataFrame(avalonTest,columns=['Col_{}'.format(i) for i in range(avalonTest.shape[1])])
avalon_fingerprintsTest

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,...,Col_502,Col_503,Col_504,Col_505,Col_506,Col_507,Col_508,Col_509,Col_510,Col_511
0,0,1,0,1,0,1,0,0,1,1,...,1,0,0,1,1,1,1,1,1,0
1,0,1,0,1,0,1,0,0,1,1,...,1,0,0,1,1,1,0,1,1,0
2,0,1,0,1,0,1,0,0,1,1,...,1,0,0,1,1,1,0,1,1,0
3,0,0,0,0,0,0,0,0,0,1,...,0,1,0,1,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
194,0,1,0,1,0,1,0,0,1,1,...,1,0,1,1,1,1,0,1,1,0
195,1,0,0,1,0,1,0,1,0,0,...,0,0,1,1,0,1,1,0,1,1
196,1,0,0,0,1,0,0,0,0,1,...,0,0,0,1,0,1,0,0,0,0


## Tính toán mô tả phân tử sử dụng Atom-Pairs fingerprint (APFP)

In [ ]:
from rdkit.Chem import rdMolDescriptors
def apfp_fingerprints(data):
    ap_fpts = []
    for i in data:
        mol = Chem.MolFromSmiles(i)
        fpts =  rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(mol, nBits=512)
        mfpts = np.array(fpts)
        ap_fpts.append(mfpts)
    return np.array(ap_fpts)

### Train Set

In [ ]:
apTrain = apfp_fingerprints(dataset_TrainNew['SMILES'])
apTrain.shape

(790, 512)

In [ ]:
apTrain

array([[1, 1, 1, ..., 1, 0, 0],
       [1, 1, 1, ..., 1, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 1, 0, 0]])

In [ ]:
# Hiển thị dạng bảng
ap_fingerprintsTrain = pd.DataFrame(apTrain,columns=['Col_{}'.format(i) for i in range(apTrain.shape[1])])
ap_fingerprintsTrain

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,...,Col_502,Col_503,Col_504,Col_505,Col_506,Col_507,Col_508,Col_509,Col_510,Col_511
0,1,1,1,0,1,1,1,0,1,1,...,0,0,1,1,1,0,1,1,0,0
1,1,1,1,1,1,1,1,0,1,1,...,0,0,1,1,1,0,1,1,0,0
2,1,1,1,1,1,0,0,0,1,1,...,1,0,1,1,1,0,1,1,1,1
3,1,1,1,1,1,1,1,0,1,1,...,1,0,1,1,1,1,1,1,1,0
4,1,1,1,0,1,1,1,0,1,1,...,1,0,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,1,0,0,0,0,0,0,0,0,0,...,1,0,1,1,1,0,1,1,1,0
786,1,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,0
787,1,1,1,0,1,1,1,1,1,1,...,0,0,1,1,0,0,1,1,1,0
788,1,1,1,0,1,1,1,1,1,1,...,1,0,1,1,1,0,1,1,1,0


### Test Set

In [ ]:
apTest = apfp_fingerprints(dataset_TestNew['SMILES'])
apTest.shape

(198, 512)

In [ ]:
apTest

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 0, 0, ..., 1, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 0]])

In [ ]:
# Hiển thị dạng bảng
ap_fingerprintsTest = pd.DataFrame(apTest,columns=['Col_{}'.format(i) for i in range(apTest.shape[1])])
ap_fingerprintsTest

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,...,Col_502,Col_503,Col_504,Col_505,Col_506,Col_507,Col_508,Col_509,Col_510,Col_511
0,1,1,1,1,1,1,1,0,1,1,...,1,0,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,0,1,1,...,0,0,1,1,1,1,1,1,1,1
3,1,1,1,0,1,1,0,0,1,1,...,0,0,1,1,0,0,1,1,0,0
4,1,1,1,0,1,1,1,0,1,1,...,0,0,1,1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,1,1,1,0,1,1,1,0,1,1,...,1,0,1,1,1,0,1,1,1,1
194,1,1,1,1,1,1,1,0,1,1,...,1,0,1,1,1,1,1,1,1,1
195,1,0,0,0,1,1,0,0,0,0,...,0,0,1,1,1,0,1,1,0,0
196,1,1,1,0,1,1,0,0,1,1,...,1,0,1,1,1,1,1,1,1,1


## Tính toán mô tả phân tử sử dụng Topological-Torsions fingerprint (TTFP)

In [ ]:
from rdkit.Chem import rdMolDescriptors
def ttfp_fingerprints(data):
    tt_fpts = []
    for i in data:
        mol = Chem.MolFromSmiles(i)
        fpts =  TT = rdMolDescriptors.GetHashedTopologicalTorsionFingerprintAsBitVect(mol, nBits=512)
        mfpts = np.array(fpts)
        tt_fpts.append(mfpts)
    return np.array(tt_fpts)

### Train Set

In [ ]:
ttTrain = ttfp_fingerprints(dataset_TrainNew['SMILES'])
ttTrain.shape

(790, 512)

In [ ]:
ttTrain

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [ ]:
# Hiển thị dạng bảng
tt_fingerprintsTrain = pd.DataFrame(ttTrain,columns=['Col_{}'.format(i) for i in range(ttTrain.shape[1])])
tt_fingerprintsTrain

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,...,Col_502,Col_503,Col_504,Col_505,Col_506,Col_507,Col_508,Col_509,Col_510,Col_511
0,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
786,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
787,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
788,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


### Test Set

In [ ]:
ttTest = ttfp_fingerprints(dataset_TestNew['SMILES'])
ttTest.shape

(198, 512)

In [ ]:
ttTest

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 1, 1],
       [1, 1, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])

In [ ]:
# Hiển thị dạng bảng
tt_fingerprintsTest = pd.DataFrame(ttTest,columns=['Col_{}'.format(i) for i in range(ttTest.shape[1])])
tt_fingerprintsTest

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,...,Col_502,Col_503,Col_504,Col_505,Col_506,Col_507,Col_508,Col_509,Col_510,Col_511
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,1,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
2,1,1,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,1,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,1,0,0
194,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
195,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


## Tính toán mô tả phân tử sử dụng Molecular ACCess System (MACCS) keys

In [ ]:
from rdkit.Chem import MACCSkeys
def maccs_fingerprints(data):
    maccs_fpts = []
    for i in data:
        mol = Chem.MolFromSmiles(i)
        fpts = MACCSkeys.GenMACCSKeys(mol)
        mfpts = np.array(fpts)
        maccs_fpts.append(mfpts)
    return np.array(maccs_fpts)

### Train Set

In [ ]:
maccsTrain = maccs_fingerprints(dataset_TrainNew['SMILES'])
maccsTrain.shape

(790, 167)

In [ ]:
maccsTrain

array([[0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0]])

In [ ]:
# Hiển thị dạng bảng
maccs_fingerprintsTrain = pd.DataFrame(maccsTrain,columns=['Col_{}'.format(i) for i in range(maccsTrain.shape[1])])
maccs_fingerprintsTrain

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,...,Col_157,Col_158,Col_159,Col_160,Col_161,Col_162,Col_163,Col_164,Col_165,Col_166
0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,1,1,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,1,1,1,1,0
3,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,1,1,1,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,1,1,1,1,0
786,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
787,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
788,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0


### Test Set

In [ ]:
maccsTest = maccs_fingerprints(dataset_TestNew['SMILES'])
maccsTest.shape

(198, 167)

In [ ]:
maccsTest

array([[0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0]])

In [ ]:
# Hiển thị dạng bảng
maccs_fingerprintsTest = pd.DataFrame(maccsTest,columns=['Col_{}'.format(i) for i in range(maccsTest.shape[1])])
maccs_fingerprintsTest

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,...,Col_157,Col_158,Col_159,Col_160,Col_161,Col_162,Col_163,Col_164,Col_165,Col_166
0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,1,1,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,1,1,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,1,1,1,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,1,1,1,1,0
194,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,1,1,1,1,0
195,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,1,0,1,1,0
196,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,1,1,1,0


# **Lấy các giá trị phân loại ban đầu**

## Train Set

In [ ]:
y_train = dataset.iloc[:, -1].values

In [ ]:
y_train

array([1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,

In [ ]:
len(y_train)

790

## Test Set

In [ ]:
y_test = dataset_test.iloc[:, -1].values

In [ ]:
y_test

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1])

In [ ]:
len(y_test)

198

# **Chọn bộ mô tả phần tử**

In [ ]:
# Mordred


In [ ]:
# RDkit
dataTrain = df_with_200_descriptorsTrain
dataTest = df_with_200_descriptorsTest

In [ ]:
#  Morgan fingerprint
dataTrain = Morgan_fptsTrain
dataTest = Morgan_fptsTest

In [ ]:
#  Avalon
dataTrain = avalonTrain
dataTest = avalonTest

In [ ]:
#  Atom Pairs
dataTrain = apTrain
dataTest = apTest

In [ ]:
#  Topological-Torsions
dataTrain = ttTrain
dataTest = ttTest

In [ ]:
#  Molecular ACCess System keys
dataTrain = maccsTrain
dataTest = maccsTest

# **Feature Scaling**

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(dataTrain)
X_test = sc.transform(dataTest)

In [ ]:
print(X_train)

[[0.         0.         0.         ... 0.         0.05037927 0.        ]
 [0.         0.         0.         ... 0.         0.05037927 0.        ]
 [0.         0.         0.         ... 0.         0.05037927 0.        ]
 ...
 [0.         0.         0.         ... 0.         0.05037927 0.        ]
 [0.         0.         0.         ... 0.         0.05037927 0.        ]
 [0.         0.         0.         ... 0.         0.05037927 0.        ]]


In [ ]:
print(X_test)

[[0.         0.         0.         ... 0.         0.05037927 0.        ]
 [0.         0.         0.         ... 0.         0.05037927 0.        ]
 [0.         0.         0.         ... 0.         0.05037927 0.        ]
 ...
 [0.         0.         0.         ... 0.         0.05037927 0.        ]
 [0.         0.         0.         ... 0.         0.05037927 0.        ]
 [0.         0.         0.         ... 0.         0.05037927 0.        ]]


# **Chọn thuật toán phân loại để xây dựng mô hình**

## Logistic Regression



In [ ]:
# Training the LR model on the Training set
# penalty: phương pháp điều chuẩn (regularization) để giảm overfitting trong mô hình => "l1" (L1 regularization) và "l2" (L2 regularization).
# C: nghịch đảo của hệ số điều chuẩn (regularization strength). C <<, điều chuẩn càng mạnh và mô hình càng đơn giản và ngược lại.
# solver: thuật toán tối ưu hóa (để tìm bộ trọng số tối ưu cho mô hình) => "lbfgs", "newton-cg", "sag" và "saga" (Mỗi thuật toán sẽ có hiệu suất và tốc độ hội tụ khác nhau tùy thuộc vào dữ liệu.)
# max_iter:  số lượng lần lặp tối đa để tìm bộ trọng số tối ưu (Nếu mô hình không hội tụ sau số lần lặp này, quá trình huấn luyện sẽ dừng lại)
# random_state: seed để tạo số ngẫu nhiên. (Khi đặt random_state thành một số nguyên, mô hình sẽ tạo ra các số ngẫu nhiên có thể tái sản xuất khi bạn chạy lại mã.)
# class_weight: trọng số cho các lớp trong trường hợp dữ liệu mất cân bằng. (Nếu không được chỉ định, mô hình sẽ giả định rằng tất cả các lớp có trọng số bằng nhau.)

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

## K-Nearest Neighbors



In [ ]:
# Training the K-NN model on the Training set
# n_neighbors: số hàng xóm
# weights: cách tính toán trọng số cho các láng giềng gần nhất khi thực hiện phân loại.
#   "uniform" (mặc định): Các láng giềng gần nhất có cùng mức độ ảnh hưởng đến dự đoán.
#   "distance": Trọng số của mỗi láng giềng gần nhất tỉ lệ nghịch với khoảng cách từ điểm dữ liệu mới đến láng giềng đó.
# metric: phương pháp đo khoảng cách "euclidean", "manhattan", "minkowski", v.v.
# p: chỉ định giá trị của tham số p trong phương pháp khoảng cách Minkowski. Khi p = 2, khoảng cách Minkowski tương đương với khoảng cách Euclidean.
# algorithm: thuật toán tính toán láng giềng gần nhất.
#   "auto" (mặc định): tự động chọn thuật toán phù hợp nhất dựa vào dữ liệu đầu vào.
#   "ball_tree": Sử dụng cấu trúc cây bóng để tìm kiếm láng giềng gần nhất.
#   "kd_tree": Sử dụng cấu trúc cây kd để tìm kiếm láng giềng gần nhất.
#   "brute": Tìm kiếm láng giềng gần nhất bằng cách duyệt tất cả các điểm dữ liệu trong tập huấn luyện.
# leaf_size: Kích thước lá của cây khi sử dụng thuật toán "ball_tree" hoặc "kd_tree".
# n_jobs: số luồng hoặc CPU sử dụng để tính toán. Nếu n_jobs = -1, mô hình sẽ sử dụng tất cả các CPU có sẵn.
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

KNeighborsClassifier()

## Support Vector Machine

In [ ]:
# Training the SVM model on the Training set
# C: Tham số điều chỉnh độ rộng của biên (margin). Giá trị C càng lớn, mô hình càng không chấp nhận sai sót (overfitting). Ngược lại, giá trị C càng nhỏ, mô hình càng chấp nhận sai sót (underfitting).
# kernel: hàm kernel được sử dụng để tính toán độ tương đồng giữa các điểm dữ liệu. => "linear" (đường thẳng), "poly" (đa thức), "rbf" (hàm Gaussian), "sigmoid" (hàm sigmoid),...
# degree: Tham số sử dụng cho hàm đa thức (kernel="poly"). (Nó xác định bậc của đa thức.)
# gamma: Tham số điều chỉnh hàm kernel. (Nó quyết định sự phụ thuộc của các điểm dữ liệu trong hàm kernel. Các giá trị gamma nhỏ sẽ tạo ra một đường cong lỏng lẻo và giá trị gamma lớn tạo ra một đường cong chặt chẽ.)
# random_state: seed để tạo số ngẫu nhiên. (Khi đặt random_state thành một số nguyên, mô hình sẽ tạo ra các số ngẫu nhiên có thể tái sản xuất khi bạn chạy lại mã.)
# class_weight: trọng số cho các lớp trong trường hợp dữ liệu mất cân bằng. (Nếu không được chỉ định, mô hình sẽ giả định rằng tất cả các lớp có trọng số bằng nhau.)

from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train, y_train)

SVC(kernel='linear', random_state=0)

## Decision Tree

In [ ]:
# Training the DT model on the Training set
# criterion: phương pháp đo lường độ tốt của một phân tách. => "gini" (chỉ số Gini) hoặc "entropy" (thang đo entropy).
# splitter: cách chọn phân tách ở mỗi nút. => "best" (chọn phân tách tốt nhất) hoặc "random" (chọn phân tách ngẫu nhiên).
# max_depth: độ sâu tối đa của cây. (Giá trị max_depth càng lớn, cây càng phức tạp và có khả năng overfitting.)
# min_samples_split: Số lượng mẫu tối thiểu yêu cầu để chia một nút thành các nút con.( Nếu số lượng mẫu trong một nút nhỏ hơn min_samples_split, cây sẽ không tiếp tục phân tách.)
# min_samples_leaf: Số lượng mẫu tối thiểu yêu cầu để một nút được coi là nút lá (không thể phân tách tiếp).( Nếu số lượng mẫu trong một nút nhỏ hơn min_samples_leaf, cây sẽ ngưng phát triển.)
# max_features: số lượng đặc trưng tối đa được cân nhắc khi tìm kiếm phân tách tốt nhất.
# random_state: seed để tạo số ngẫu nhiên. (Khi đặt random_state thành một số nguyên, mô hình sẽ tạo ra các số ngẫu nhiên có thể tái sản xuất khi bạn chạy lại mã.)

from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

## Random Forest

In [ ]:
# Training the Random Forest Classification model on the Training set
# n_estimators: số lượng cây quyết định trong ensemble.
# criterion: đo lường chất lượng của việc chia dữ liệu trong quá trình xây dựng cây. => "gini" và "entropy". "gini" sử dụng chỉ số Gini và "entropy" sử dụng thông số Entropy để đo lường.
# max_depth: độ sâu tối đa của các cây quyết định trong ensemble.
# min_samples_split: Số lượng mẫu tối thiểu yêu cầu để chia một nút trong cây quyết định. ( kiểm soát độ phức tạp và overfitting của mô hình.)
# min_samples_leaf: Số lượng mẫu tối thiểu yêu cầu để tạo một lá cây. (kiểm soát overfitting.)
# random_state: seed để tạo số ngẫu nhiên. (Khi đặt random_state thành một số nguyên, mô hình sẽ tạo ra các số ngẫu nhiên có thể tái sản xuất khi bạn chạy lại mã.)
# class_weight: trọng số cho các lớp trong trường hợp dữ liệu mất cân bằng. (Nếu không được chỉ định, mô hình sẽ giả định rằng tất cả các lớp có trọng số bằng nhau.)

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', random_state=0)

## XGBoosting

In [ ]:
# Training the XGBoosting model on the Training set
# max_depth: Độ sâu tối đa của cây quyết định. (Điều này giới hạn độ sâu của cây để tránh overfitting. Một giá trị thường được sử dụng là 3-10.)
# learning_rate: Tỷ lệ học tập. (quy định mức độ cập nhật trọng số sau mỗi bước học tập. Một giá trị thấp có thể giúp mô hình học chậm hơn và dễ dàng hội tụ, trong khi một giá trị cao có thể làm cho mô hình học nhanh hơn nhưng có thể dẫn đến overfitting.)
# n_estimators: Số lượng cây quyết định trong ensemble. (ảnh hưởng đến số lượng bước học tập được thực hiện và thời gian huấn luyện.)
# subsample: Tỷ lệ mẫu con được sử dụng để huấn luyện mỗi cây. (Giá trị thường được sử dụng là 0.5-1.0. Giá trị nhỏ hơn 1.0 sẽ giảm khả năng overfitting.)
# colsample_bytree: Tỷ lệ số lượng cột con được sử dụng trong việc xây dựng mỗi cây. (Giá trị thường được sử dụng là 0.5-1.0. Giá trị nhỏ hơn 1.0 giúp giảm overfitting.)
# gamma: Giá trị cắt tỉa của cây. (Giá trị này quy định mức tăng của giá trị loss function mà mô hình sẽ dừng lại và cắt tỉa cây. Giá trị thường được sử dụng là 0.1-0.3.)
# reg_alpha và reg_lambda: Tham số điều chuẩn L1 và L2. (Các giá trị dương giúp giảm overfitting bằng cách giới hạn trọng số của các biến độc lập.)
# objective: Hàm mục tiêu mà mô hình cố gắng tối ưu hóa. (bài toán phân loại nhị phân → 'binary:logistic'. Bài toán phân loại đa lớp → 'multi:softmax' và chỉ định số lớp thông qua tham số num_class.)
# random_state: đảm bảo kết quả có thể tái lặp, bạn có thể đặt một giá trị nguyên cho random_state.

from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

## Deep Neutral Network

In [ ]:
# import thư viện

import numpy as np
from keras.models import Sequential
from keras.layers import Dense


In [ ]:
# Khởi tạo mô hình bằng cách tạo một đối tượng của lớp Sequential:
model = Sequential()

# Thêm lớp Dense đầu tiên với số lượng đơn vị (neuron) là 32 và hàm kích hoạt là 'relu'
model.add(Dense(32, activation='relu', input_dim=512))
# Thêm các lớp Dense tiếp theo với số lượng đơn vị và hàm kích hoạt tùy ý
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))

# Thêm lớp đầu ra với số lượng đơn vị bằng số lượng lớp đầu ra và hàm kích hoạt là 'softmax' (đối với bài toán phân loại)
model.add(Dense(1, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test))

accuracy = model.evaluate(X_test, y_test)[1]
print("Accuracy on test set:", accuracy)




Epoch 1/50


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


25/25 [==============================] - 1s 11ms/step - loss: 0.0000e+00 - accuracy: 0.6747 - val_loss: 0.0000e+00 - val_accuracy: 0.7020
Epoch 2/50
25/25 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.6747 - val_loss: 0.0000e+00 - val_accuracy: 0.7020
Epoch 3/50
25/25 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.6747 - val_loss: 0.0000e+00 - val_accuracy: 0.7020
Epoch 4/50
25/25 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.6747 - val_loss: 0.0000e+00 - val_accuracy: 0.7020
Epoch 5/50
25/25 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.6747 - val_loss: 0.0000e+00 - val_accuracy: 0.7020
Epoch 6/50
25/25 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.6747 - val_loss: 0.0000e+00 - val_accuracy: 0.7020
Epoch 7/50
25/25 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.6747 - val_loss: 0.0000e+

## Kernel Support Vector Machine

In [ ]:
# Training the SVM model on the Training set
# C: Tham số điều chỉnh độ rộng của biên (margin). Giá trị C càng lớn, mô hình càng không chấp nhận sai sót (overfitting). Ngược lại, giá trị C càng nhỏ, mô hình càng chấp nhận sai sót (underfitting).
# kernel: hàm kernel được sử dụng để tính toán độ tương đồng giữa các điểm dữ liệu. => "linear" (đường thẳng), "poly" (đa thức), "rbf" (hàm Gaussian), "sigmoid" (hàm sigmoid),...
# degree: Tham số sử dụng cho hàm đa thức (kernel="poly"). (Nó xác định bậc của đa thức.)
# gamma: Tham số điều chỉnh hàm kernel. (Nó quyết định sự phụ thuộc của các điểm dữ liệu trong hàm kernel. Các giá trị gamma nhỏ sẽ tạo ra một đường cong lỏng lẻo và giá trị gamma lớn tạo ra một đường cong chặt chẽ.)
# random_state: seed để tạo số ngẫu nhiên. (Khi đặt random_state thành một số nguyên, mô hình sẽ tạo ra các số ngẫu nhiên có thể tái sản xuất khi bạn chạy lại mã.)
# class_weight: trọng số cho các lớp trong trường hợp dữ liệu mất cân bằng. (Nếu không được chỉ định, mô hình sẽ giả định rằng tất cả các lớp có trọng số bằng nhau.)

from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)

SVC(random_state=0)

## Naive Bayes

In [ ]:
# Training the Naive Bayes model on the Training set
# không có siêu tham số

from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

# **Predict and Calc Confusion Matrix**

In [ ]:
y_pred = classifier.predict(X_test)
# print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 44  15]
 [ 13 126]]


0.8585858585858586